In [10]:
import openeo
import geopandas as gpd
import leafmap

In [11]:
eoconn = openeo.connect("https://openeo-dev.vito.be")
eoconn.authenticate_oidc()
eoconn.describe_account()

Visit https://aai.egi.eu/device?user_code=VUWO-VCCA 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


{'info': {'oidc_userinfo': {'eduperson_assurance': ['https://refeds.org/assurance/IAP/low',
    'https://aai.egi.eu/LoA#Substantial'],
   'eduperson_entitlement': ['urn:mace:egi.eu:group:vo.openeo.cloud:role=member#aai.egi.eu',
    'urn:mace:egi.eu:group:vo.openeo.cloud:role=vm_operator#aai.egi.eu',
    'urn:mace:egi.eu:group:vo.pangeo.eu:role=member#aai.egi.eu',
    'urn:mace:egi.eu:group:vo.pangeo.eu:role=vm_operator#aai.egi.eu'],
   'eduperson_scoped_affiliation': ['employee@eurac.edu', 'member@eurac.edu'],
   'email': 'valentina.premier@eurac.edu',
   'email_verified': True,
   'sub': 'b5345703744f82d43e36ecfbbe263186fb2556417bd8fe0054ae63476450f901@egi.eu',
   'voperson_verified_email': ['Valentina.Premier@eurac.edu']}},
 'name': 'Valentina.Premier@eurac.edu',
 'user_id': 'b5345703744f82d43e36ecfbbe263186fb2556417bd8fe0054ae63476450f901@egi.eu'}

Connect and Authenticate

In [2]:
eoconn = openeo.connect('https://openeo.dataspace.copernicus.eu/', auto_validate=False)
eoconn.authenticate_oidc()

eoconn.describe_account()

Authenticated using refresh token.


{'info': {'oidc_userinfo': {'email': 'valentina.premier@eurac.edu',
   'email_verified': True,
   'family_name': 'Premier',
   'given_name': 'Valentina',
   'name': 'Valentina Premier',
   'preferred_username': 'valentina.premier@eurac.edu',
   'sub': '1aa67f75-e178-495e-a131-9f7dffaaae18'}},
 'name': 'Valentina Premier',
 'user_id': '1aa67f75-e178-495e-a131-9f7dffaaae18'}

Load a shapefile and define the area of interest

In [3]:
shp_path = r'/mnt/CEPH_PROJECTS/ALPSNOW/Nicola/Shapefile/Hydrological_basins/Fleres_shape/Fleres_4326.shp'

catchment_outline = gpd.read_file(shp_path)
bbox = catchment_outline.bounds.iloc[0]

center = (float(catchment_outline.centroid.y), 
          float(catchment_outline.centroid.x))
m = leafmap.Map(center=center, zoom=1)
m.add_vector(shp_path, layer_name="catchment")
m

Map(center=[46.95421445230168, 11.348608981253069], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [4]:
bbox

minx    11.241593
miny    46.914255
maxx    11.444267
maxy    46.992429
Name: 0, dtype: float64

Load the Copernicus Global 30 meter Digital Elevation Model (DEM) dataset.

In [6]:
dem_datacube = eoconn.load_collection(
    "COPERNICUS_30",
    spatial_extent={'west':bbox[0],
                    'east':bbox[2],
                    'south':bbox[1],
                    'north':bbox[3],
                    'crs':4326},
    bands=["DEM"]
)
#dem_datacube.download('./results/dem.nc')

In [7]:
#dem_datacube=dem_datacube.drop_dimension("t")
dem_datacube=dem_datacube.drop_dimension("bands")

#dem_datacube=dem_datacube.reduce_dimension(reducer="mean",dimension="t")
#dem_datacube=dem_datacube.reduce_dimension(reducer="mean",dimension="bands")

Run the udf to get the hillshade. Use apply neighborhood to avoid missing data that appear at the edges of the chunks and ensure continuity in the output.

In [9]:
# Load the UDF from a file.
udf_path="./udf-hillshade.py"

hillshade_func = openeo.UDF.from_file(udf_path, context={"from_parameter": "context"})


# Apply the UDF to the data cube.
hillshade = dem_datacube.apply_neighborhood(
    hillshade_func,
    size=[
        {"dimension": "x", "value": 224, "unit": "px"},
        {"dimension": "y", "value": 224, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 16, "unit": "px"},
        {"dimension": "y", "value": 16, "unit": "px"}
    ],
)
hillshade.download('./results/hillshade.tif')
